In [10]:
import pandas as pd

dataset = pd.read_csv('./compounds_select_1000_ans.csv', encoding='utf8')
dataset[dataset.iloc[:, 2] == 2.].head(10)

,Часть 1,Часть 2,Класс композициональности,Контекст 1,Контекст 2,Контекст 3,Контекст 4,Контекст 5,Контекст 6,Контекст 7,...,Контекст 241,Контекст 242,Контекст 243,Контекст 244,Контекст 245,Контекст 246,Контекст 247,Контекст 248,Контекст 249,Контекст 250
119,активность,масс,2.0,Все это осуществляется на волне пробудившейся ...,"С другой стороны , имеется в виду поднять экон...","Наряду с обновлением экономической , социально...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,год,перестройки,2.0,"В этом я вижу главную причину того , что три г...","И вот сегодня , за три года перестройки , на н...","Поэтому в годы перестройки , когда разоблачали...",Структура расходов российского бюджета за годы...,Семью Олега Антонова трудно удивить высокими ц...,Поддерживали друзья и соратники по годам перес...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,лишение,прав,2.0,Судилась с 21 февраля 1877 года и 14 марта 187...,Зато графу о лишении родительских прав заполня...,24 февраля 1920 года на митинге Немецкой рабоч...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514,крупная,сеть,2.0,Именно в это время крупнейшие торговые сети на...,Системы автоматизации начали внедряться практи...,На данный момент только в одной Москве насчиты...,"Второе : договор касался не всех магазинов , а...","Управляющая компания "" Альфа-Капитал "" объявля...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545,новая,пирамида,2.0,"Учитывая все эти и многие другие детали , учен...","В новую пирамиду введено широкое основание , н...","Уже есть первые данные о здоровье людей , пита...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
564,новое,явление,2.0,Интернет - маркетинг -- явление относительно н...,"Среди совершенно новых явлений , обнаруженных ...","Экстернат - явление не новое для России , если...",Это совершенно новое явление в комсомоле .\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579,большая,возможность,2.0,"Нацизм , показывает Али , действительно обеспе...","На той же самой неделе , когда я получил LSE E...",Зато другая значительная часть экспертов ( 44 ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,православная,церковь,2.0,29 июля 1994 года решением Священного Синода У...,В XVIII веке центр сербского национально - осв...,Председатель синодального отдела по взаимодейс...,Позиция РПЦ в отношении инославия сформулирова...,Наступает лето 1988 года - юбилейного года для...,Крещение в 988 году было для Русской православ...,Святым делом назвал настоятель Богоявленского ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,советское,время,2.0,Создание казачьих частей возглавил бывший полк...,"В первой и единственной пока книге , вышедшей ...",Гайдар в советское время не был и определенно ...,"Как бы то ни было , начиная еще с советских вр...",Еще в советское время он мысленно перебирал оп...,"Это было оправданно в советские времена , когд...",Идея проведения гонок Формулы - 1 в России воз...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,открытая,дверь,2.0,"А некоторые исследователи считают , что атлант...",В 1922 году Япония согласилась ограничить тонн...,В соседней комнате через открытую дверь были в...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
parse_path = './workdir/parse1/'
result = []
for sent in GeneratorSentences(parse_path):
    result.append(sent)
    if len(result) >= 10000:
        break

In [1]:
import os
import sys
from train_dsm import train_dsm
from multiprocessing import Process
from gensim.models import FastText

logs_path = './workdir/logs/subprocesses/'


def init_subprocess_logging():
    import sys
    sys.stdout = open(os.path.join(logs_path, str(os.getpid()) + ".stdout"), "a")
    sys.stderr = open(os.path.join(logs_path, str(os.getpid()) + ".stderr"), "a")
    
    
def run_custom(func, *args, **kwargs):
    init_subprocess_logging()
    return func(*args, **kwargs)


def get_logs(proc, tp='.stderr', tail=1000):
    proc_logs_path = os.path.join(logs_path, str(proc.pid) + tp)
    with open(proc_logs_path, 'rb') as f:
        try:
            f.seek(-tail, os.SEEK_END)
        except OSError:
            pass
        finally:
            log = f.read()
    
    return log.decode('utf-8')

In [2]:
compounds_path = './workdir/annotation_katya_ref_v2.csv'
data_path = './workdir/parse5'
save_path = './workdir/models/model_fast2vec_300_5'
model = FastText(size=300, min_count=5, workers=10)
proc = Process(target=run_custom, 
               args=[train_dsm],
               kwargs=dict(compounds_path=compounds_path, 
                                    data_path=data_path, 
                                    save_path=save_path, 
                                    model=model, 
                                    epochs=5))
proc.start()

In [5]:
print(get_logs(proc))

In [3]:
proc.is_alive()

False